⚡ Intermédiaire | ⏱ 45 min | 🔑 Concepts : pygame.mixer, Sound, music, channels

# Sons et Musique

## Objectifs

- Initialiser et configurer le module audio de Pygame
- Charger et jouer des sons courts (effets sonores)
- Gérer la musique de fond (lecture, pause, volume)
- Comprendre le système de canaux audio
- Connaître les formats audio supportés et leurs limitations

## Prérequis

- Notions de base de Pygame (notebooks 01-04)
- Compréhension de la boucle de jeu

> **Note importante** : Les sons ne peuvent pas être joués directement dans Jupyter Notebook. Tous les exemples utilisent `%%writefile` pour créer des fichiers `.py` exécutables séparément. Pour tester les exemples, les fichiers audio doivent exister sur votre disque. Les démos génèrent des sons procéduraux quand c'est possible.

## 1. pygame.mixer - Initialisation et Configuration

Le module `pygame.mixer` gère tout l'audio dans Pygame : sons courts et musique de fond.

### 1.1 Initialisation automatique

`pygame.init()` initialise automatiquement le mixer avec les paramètres par défaut :

```python
pygame.init()  # Initialise aussi le mixer
```

### 1.2 Initialisation personnalisée avec pre_init()

Pour contrôler la qualité audio, utilisez `pre_init()` **avant** `pygame.init()` :

```python
# Configurer AVANT pygame.init()
pygame.mixer.pre_init(
    frequency=44100,  # Taux d'échantillonnage (Hz)
    size=-16,          # Taille des échantillons (16 bits signé)
    channels=2,        # Stéréo
    buffer=512         # Taille du buffer (petit = moins de latence)
)
pygame.init()
```

### Paramètres de pre_init()

| Paramètre | Défaut | Description |
|---|---|---|
| `frequency` | 44100 | Qualité audio (22050 = FM, 44100 = CD, 48000 = DVD) |
| `size` | -16 | Bits par échantillon (-16 = 16 bits signé, standard) |
| `channels` | 2 | 1 = mono, 2 = stéréo |
| `buffer` | 512 | Taille du buffer (puissance de 2, petit = réactif mais CPU) |

### 1.3 Vérifier l'initialisation

```python
# Vérifier si le mixer est initialisé
if pygame.mixer.get_init():
    freq, size, channels = pygame.mixer.get_init()
    print(f"Fréquence: {freq} Hz, Taille: {size}, Canaux: {channels}")
```

## 2. Sons Courts - pygame.mixer.Sound

Les objets `Sound` sont utilisés pour les **effets sonores** : tirs, explosions, sauts, collecte d'objets...

### 2.1 Charger un son

```python
# Depuis un fichier
son_tir = pygame.mixer.Sound("tir.wav")
son_explosion = pygame.mixer.Sound("explosion.ogg")

# Depuis un buffer (son procédural)
import numpy as np
# Générer un bip de 440 Hz
freq = 440
duree = 0.2  # secondes
sample_rate = 44100
t = np.linspace(0, duree, int(sample_rate * duree), False)
signal = np.sin(2 * np.pi * freq * t) * 32767
son_bip = pygame.mixer.Sound(signal.astype(np.int16))
```

### 2.2 Jouer un son

```python
# Jouer une fois
son_tir.play()

# Jouer en boucle (-1 = infini, n = nombre de répétitions)
son_moteur.play(loops=-1)  # Boucle infinie
son_alerte.play(loops=3)   # Jouer 4 fois (1 + 3 boucles)

# Jouer avec une durée maximale
son_tir.play(maxtime=500)  # Maximum 500 ms

# Jouer avec un fondu au démarrage
son_tir.play(fade_ms=200)  # Fondu de 200 ms au début
```

### 2.3 Contrôler un son

```python
# Arrêter le son
son_tir.stop()

# Volume (0.0 à 1.0)
son_tir.set_volume(0.5)  # 50%
volume = son_tir.get_volume()

# Fondu de sortie
son_tir.fadeout(1000)  # Fondu de 1 seconde

# Durée du son
duree = son_tir.get_length()  # En secondes
```

## 3. Musique de Fond - pygame.mixer.music

Le module `pygame.mixer.music` est **séparé** de `Sound`. Il est conçu pour la musique de fond (un seul morceau à la fois, streaming depuis le fichier).

### 3.1 Charger et jouer

```python
# Charger un fichier musical
pygame.mixer.music.load("musique.ogg")

# Jouer
pygame.mixer.music.play()       # Une fois
pygame.mixer.music.play(-1)     # En boucle infinie
pygame.mixer.music.play(3)      # 4 fois (1 + 3)

# Jouer avec un fondu
pygame.mixer.music.play(-1, fade_ms=3000)  # Fondu de 3s au démarrage

# Jouer à partir d'une position
pygame.mixer.music.play(-1, start=30.0)  # Commencer à 30s
```

### 3.2 Contrôles de lecture

```python
# Pause et reprise
pygame.mixer.music.pause()
pygame.mixer.music.unpause()

# Arrêt
pygame.mixer.music.stop()

# Fondu de sortie
pygame.mixer.music.fadeout(2000)  # Fondu de 2s

# Volume (0.0 à 1.0)
pygame.mixer.music.set_volume(0.7)
volume = pygame.mixer.music.get_volume()

# État de la lecture
en_lecture = pygame.mixer.music.get_busy()  # True si en cours de lecture
```

### 3.3 File d'attente

```python
# Mettre un morceau en file d'attente (joué après le morceau courant)
pygame.mixer.music.queue("morceau_suivant.ogg")

# Détecter la fin d'un morceau
MUSIQUE_FIN = pygame.USEREVENT + 1
pygame.mixer.music.set_endevent(MUSIQUE_FIN)

# Dans la boucle d'événements
for event in pygame.event.get():
    if event.type == MUSIQUE_FIN:
        # Charger le morceau suivant
        pygame.mixer.music.load("prochain_morceau.ogg")
        pygame.mixer.music.play()
```

### Différence Sound vs music

| Caractéristique | Sound | music |
|---|---|---|
| Usage | Effets sonores courts | Musique de fond |
| Chargement | Entièrement en mémoire | Streaming (économie mémoire) |
| Simultanéité | Plusieurs en même temps | Un seul à la fois |
| Contrôle | Par objet Sound | Fonctions globales |
| Formats | WAV, OGG | WAV, OGG, MP3 |

## 4. Channels (Canaux Audio)

Les canaux sont les "pistes" sur lesquelles les sons sont joués. Par défaut, Pygame dispose de 8 canaux.

### 4.1 Nombre de canaux

```python
# Obtenir le nombre de canaux
nb = pygame.mixer.get_num_channels()  # 8 par défaut

# Modifier le nombre de canaux
pygame.mixer.set_num_channels(16)  # 16 canaux
```

### 4.2 Utiliser un canal spécifique

```python
# Obtenir un canal
canal = pygame.mixer.Channel(0)  # Canal 0

# Jouer un son sur ce canal
canal.play(son_tir)

# Volume du canal
canal.set_volume(0.5)       # Mono : un seul volume
canal.set_volume(1.0, 0.0)  # Stéréo : gauche 100%, droite 0%

# Vérifier si le canal est occupé
if canal.get_busy():
    print("Canal en cours de lecture")

# Arrêter le canal
canal.stop()

# Pause et reprise
canal.pause()
canal.unpause()
```

### 4.3 Réserver des canaux

```python
# Réserver les 2 premiers canaux (ne seront pas utilisés par play() automatique)
pygame.mixer.set_reserved(2)

# Canal 0 = musique d'ambiance, Canal 1 = voix
canal_ambiance = pygame.mixer.Channel(0)
canal_voix = pygame.mixer.Channel(1)
```

### 4.4 Trouver un canal libre

```python
# Trouver un canal libre
canal = pygame.mixer.find_channel()  # None si tous occupés

# Forcer la libération du canal le plus ancien
canal = pygame.mixer.find_channel(True)  # True = forcer
```

## 5. Formats Audio Supportés

### WAV (Waveform Audio)

- **Support** : Excellent, natif
- **Qualité** : Non compressé, haute qualité
- **Taille** : Gros fichiers
- **Usage** : Effets sonores courts
- **Recommandé** pour les `Sound`

### OGG (Ogg Vorbis)

- **Support** : Excellent
- **Qualité** : Compressé, bonne qualité
- **Taille** : Fichiers compacts
- **Usage** : Musique et sons longs
- **Recommandé** pour `mixer.music`

### MP3

- **Support** : Variable selon les plateformes
- **Problèmes connus** :
  - Latence au démarrage
  - Certains fichiers MP3 ne se chargent pas
  - Boucle imparfaite (gap entre les répétitions)
- **Recommandation** : **Éviter** si possible, préférer OGG

### Conversion de formats

```bash
# Avec ffmpeg (outil en ligne de commande)
ffmpeg -i musique.mp3 musique.ogg
ffmpeg -i effet.mp3 effet.wav

# Avec Audacity (interface graphique) :
# Fichier -> Exporter -> Exporter en OGG
```

## 6. Bonnes Pratiques

### 6.1 Préchargement des sons

```python
# ❌ MAUVAIS : charger pendant le jeu
while running:
    if tirer:
        son = pygame.mixer.Sound("tir.wav")  # Lent!
        son.play()

# ✅ BON : charger au démarrage
sons = {
    'tir': pygame.mixer.Sound("tir.wav"),
    'explosion': pygame.mixer.Sound("explosion.wav"),
    'saut': pygame.mixer.Sound("saut.wav"),
}

while running:
    if tirer:
        sons['tir'].play()  # Instantané!
```

### 6.2 Gestion centralisée du volume

```python
class GestionnaireAudio:
    """Classe pour gérer tous les sons du jeu"""
    def __init__(self):
        self.sons = {}
        self.volume_global = 1.0
        self.volume_musique = 0.5
        self.volume_effets = 0.8
    
    def charger_son(self, nom, chemin):
        self.sons[nom] = pygame.mixer.Sound(chemin)
        self.sons[nom].set_volume(self.volume_effets * self.volume_global)
    
    def jouer(self, nom):
        if nom in self.sons:
            self.sons[nom].play()
    
    def set_volume_global(self, volume):
        self.volume_global = volume
        for son in self.sons.values():
            son.set_volume(self.volume_effets * self.volume_global)
        pygame.mixer.music.set_volume(self.volume_musique * self.volume_global)
```

### 6.3 Pool de sons (éviter la saturation)

```python
# Limiter le nombre de sons simultanés du même type
class PoolSons:
    def __init__(self, chemin, max_simultanes=3):
        self.sons = [pygame.mixer.Sound(chemin) for _ in range(max_simultanes)]
        self.index = 0
    
    def jouer(self):
        self.sons[self.index].play()
        self.index = (self.index + 1) % len(self.sons)

# Utilisation
pool_tir = PoolSons("tir.wav", max_simultanes=4)
pool_tir.jouer()  # Joue sur le prochain son du pool
```

In [ ]:
%%writefile demo_sons.py
"""Démonstration : Tableau de sons interactif
Chaque touche du clavier déclenche un son différent.
Les sons sont générés procéduralement (pas besoin de fichiers audio).
"""
import pygame
import sys
import math
import array

# Initialiser avec un buffer petit pour réduire la latence
pygame.mixer.pre_init(frequency=44100, size=-16, channels=1, buffer=512)
pygame.init()

LARGEUR, HAUTEUR = 900, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Tableau de Sons - Appuyez sur les touches!")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 32)
font_small = pygame.font.Font(None, 24)

# Couleurs
NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
GRIS = (60, 60, 60)


def generer_son_sinus(frequence, duree, volume=0.5):
    """Générer un son sinusoïdal procédural"""
    sample_rate = 44100
    nb_echantillons = int(sample_rate * duree)
    buf = array.array('h')  # Tableau d'entiers 16 bits signés
    amplitude = int(32767 * volume)
    
    for i in range(nb_echantillons):
        t = i / sample_rate
        # Enveloppe ADSR simple (attaque/déclin)
        enveloppe = 1.0
        if t < 0.01:  # Attaque (10 ms)
            enveloppe = t / 0.01
        elif t > duree - 0.05:  # Relâchement (50 ms)
            enveloppe = (duree - t) / 0.05
        
        valeur = int(amplitude * math.sin(2 * math.pi * frequence * t) * enveloppe)
        buf.append(max(-32767, min(32767, valeur)))
    
    return pygame.mixer.Sound(buffer=buf)


def generer_bruit(duree, volume=0.3):
    """Générer un bruit blanc (pour percussion)"""
    import random
    sample_rate = 44100
    nb_echantillons = int(sample_rate * duree)
    buf = array.array('h')
    amplitude = int(32767 * volume)
    
    for i in range(nb_echantillons):
        t = i / sample_rate
        enveloppe = max(0, 1 - t / duree)  # Déclin linéaire
        valeur = int(random.randint(-amplitude, amplitude) * enveloppe)
        buf.append(valeur)
    
    return pygame.mixer.Sound(buffer=buf)


# Générer les sons pour chaque touche (gamme de Do)
# Do, Ré, Mi, Fa, Sol, La, Si, Do+
notes = {
    pygame.K_a: ('Do', 261.63, (255, 80, 80)),
    pygame.K_z: ('Ré', 293.66, (255, 160, 80)),
    pygame.K_e: ('Mi', 329.63, (255, 255, 80)),
    pygame.K_r: ('Fa', 349.23, (80, 255, 80)),
    pygame.K_t: ('Sol', 392.00, (80, 255, 255)),
    pygame.K_y: ('La', 440.00, (80, 80, 255)),
    pygame.K_u: ('Si', 493.88, (200, 80, 255)),
    pygame.K_i: ('Do+', 523.25, (255, 80, 200)),
}

# Sons spéciaux
sons_speciaux = {
    pygame.K_SPACE: ('Percussion', None, (200, 200, 200)),
    pygame.K_1: ('Grave', 130.81, (150, 80, 80)),
    pygame.K_2: ('Aigu', 1046.50, (80, 80, 150)),
}

# Créer les objets Sound
sons = {}
for touche, (nom, freq, couleur) in notes.items():
    sons[touche] = generer_son_sinus(freq, 0.4, 0.4)

for touche, (nom, freq, couleur) in sons_speciaux.items():
    if freq is None:
        sons[touche] = generer_bruit(0.15, 0.3)
    else:
        sons[touche] = generer_son_sinus(freq, 0.3, 0.4)

# État des touches (pour l'affichage visuel)
touches_actives = {}  # touche -> timer

# Volume global
volume_global = 0.7

# ========== BOUCLE PRINCIPALE ==========
running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            
            # Jouer le son correspondant
            if event.key in sons:
                sons[event.key].set_volume(volume_global)
                sons[event.key].play()
                touches_actives[event.key] = 0.3  # Animation de 0.3s
            
            # Contrôle du volume
            if event.key == pygame.K_UP:
                volume_global = min(1.0, volume_global + 0.1)
            if event.key == pygame.K_DOWN:
                volume_global = max(0.0, volume_global - 0.1)
    
    # Mettre à jour les timers des touches actives
    touches_a_supprimer = []
    for touche, timer in touches_actives.items():
        touches_actives[touche] -= dt
        if touches_actives[touche] <= 0:
            touches_a_supprimer.append(touche)
    for touche in touches_a_supprimer:
        del touches_actives[touche]
    
    # ---- Affichage ----
    ecran.fill(NOIR)
    
    # Titre
    titre = font.render("Tableau de Sons Interactif", True, BLANC)
    ecran.blit(titre, (LARGEUR // 2 - titre.get_width() // 2, 20))
    
    # Afficher les notes
    x_start = 50
    y_notes = 120
    largeur_touche = 90
    hauteur_touche = 100
    
    toutes_touches = list(notes.items())
    for i, (touche, (nom, freq, couleur)) in enumerate(toutes_touches):
        x = x_start + i * (largeur_touche + 10)
        
        # Couleur active ou gris
        if touche in touches_actives:
            couleur_fond = couleur
        else:
            couleur_fond = GRIS
        
        # Dessiner la touche
        pygame.draw.rect(ecran, couleur_fond, (x, y_notes, largeur_touche, hauteur_touche),
                        border_radius=8)
        pygame.draw.rect(ecran, BLANC, (x, y_notes, largeur_touche, hauteur_touche),
                        width=2, border_radius=8)
        
        # Nom de la note
        texte_nom = font.render(nom, True, BLANC)
        ecran.blit(texte_nom, (x + largeur_touche // 2 - texte_nom.get_width() // 2,
                               y_notes + 20))
        
        # Touche à presser
        nom_touche = pygame.key.name(touche).upper()
        texte_touche = font_small.render(f"[{nom_touche}]", True, BLANC)
        ecran.blit(texte_touche, (x + largeur_touche // 2 - texte_touche.get_width() // 2,
                                  y_notes + 65))
    
    # Sons spéciaux
    y_special = 300
    x_special = 50
    for i, (touche, (nom, freq, couleur)) in enumerate(sons_speciaux.items()):
        x = x_special + i * 200
        
        if touche in touches_actives:
            couleur_fond = couleur
        else:
            couleur_fond = GRIS
        
        pygame.draw.rect(ecran, couleur_fond, (x, y_special, 170, 70), border_radius=8)
        pygame.draw.rect(ecran, BLANC, (x, y_special, 170, 70), width=2, border_radius=8)
        
        nom_touche = pygame.key.name(touche).upper()
        texte = font.render(f"{nom}", True, BLANC)
        texte2 = font_small.render(f"[{nom_touche}]", True, BLANC)
        ecran.blit(texte, (x + 85 - texte.get_width() // 2, y_special + 10))
        ecran.blit(texte2, (x + 85 - texte2.get_width() // 2, y_special + 42))
    
    # Barre de volume
    y_vol = 450
    texte_vol = font.render(f"Volume: {int(volume_global * 100)}%", True, BLANC)
    ecran.blit(texte_vol, (50, y_vol))
    
    # Barre de progression du volume
    pygame.draw.rect(ecran, GRIS, (250, y_vol + 5, 300, 20))
    pygame.draw.rect(ecran, (80, 200, 80), (250, y_vol + 5, int(300 * volume_global), 20))
    pygame.draw.rect(ecran, BLANC, (250, y_vol + 5, 300, 20), 2)
    
    # Instructions
    instructions = [
        "A Z E R T Y U I : notes de musique (Do à Do+)",
        "ESPACE: percussion | 1: grave | 2: aigu",
        "Haut/Bas: volume | ESC: quitter",
    ]
    y_inst = HAUTEUR - 100
    for instruction in instructions:
        texte = font_small.render(instruction, True, (180, 180, 180))
        ecran.blit(texte, (50, y_inst))
        y_inst += 28
    
    # Info mixer
    freq_mixer, size_mixer, channels_mixer = pygame.mixer.get_init()
    info = font_small.render(
        f"Mixer: {freq_mixer}Hz | {abs(size_mixer)}bit | "
        f"{'Stéréo' if channels_mixer == 2 else 'Mono'} | "
        f"Canaux: {pygame.mixer.get_num_channels()}",
        True, (100, 100, 100)
    )
    ecran.blit(info, (10, HAUTEUR - 25))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

In [ ]:
%%writefile demo_musique.py
"""Démonstration : Lecteur de musique simple
Contrôles : play, pause, stop, volume, progression.
Génère une mélodie procédurale si aucun fichier audio n'est disponible.
"""
import pygame
import sys
import math
import array
import os

pygame.mixer.pre_init(frequency=44100, size=-16, channels=1, buffer=1024)
pygame.init()

LARGEUR, HAUTEUR = 800, 500
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Lecteur de Musique")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)
font_small = pygame.font.Font(None, 24)

# Couleurs
NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
GRIS = (80, 80, 80)
GRIS_CLAIR = (150, 150, 150)
VERT = (80, 200, 80)
ROUGE = (200, 80, 80)
BLEU = (80, 120, 200)
JAUNE = (200, 200, 80)


def generer_melodie(duree_totale=30):
    """Générer une mélodie simple en mémoire"""
    sample_rate = 44100
    buf = array.array('h')
    
    # Mélodie simple : suite de notes
    notes_melodie = [
        (261.63, 0.5), (293.66, 0.5), (329.63, 0.5), (349.23, 0.5),
        (392.00, 1.0), (349.23, 0.5), (329.63, 0.5),
        (293.66, 1.0), (261.63, 1.0),
        (329.63, 0.5), (349.23, 0.5), (392.00, 0.5), (440.00, 0.5),
        (523.25, 1.0), (440.00, 0.5), (392.00, 0.5),
        (349.23, 1.0), (329.63, 1.0),
    ]
    
    temps_courant = 0
    index_note = 0
    nb_echantillons = int(sample_rate * duree_totale)
    
    for i in range(nb_echantillons):
        t = i / sample_rate
        
        # Déterminer la note courante
        freq, duree_note = notes_melodie[index_note % len(notes_melodie)]
        t_dans_note = t - temps_courant
        
        if t_dans_note >= duree_note:
            temps_courant += duree_note
            index_note += 1
            freq, duree_note = notes_melodie[index_note % len(notes_melodie)]
            t_dans_note = t - temps_courant
        
        # Enveloppe
        ratio = t_dans_note / duree_note
        if ratio < 0.05:
            env = ratio / 0.05
        elif ratio > 0.7:
            env = (1 - ratio) / 0.3
        else:
            env = 1.0
        
        # Signal
        valeur = int(12000 * math.sin(2 * math.pi * freq * t) * env)
        buf.append(max(-32767, min(32767, valeur)))
    
    return pygame.mixer.Sound(buffer=buf)


# Générer la musique procédurale
print("Génération de la mélodie...")
melodie = generer_melodie(30)
duree_totale = melodie.get_length()
print(f"Mélodie générée : {duree_totale:.1f} secondes")

# État du lecteur
en_lecture = False
en_pause = False
volume = 0.7
temps_lecture = 0
canal = None

# Boutons
class Bouton:
    def __init__(self, x, y, largeur, hauteur, texte, couleur):
        self.rect = pygame.Rect(x, y, largeur, hauteur)
        self.texte = texte
        self.couleur = couleur
        self.survol = False
    
    def dessiner(self, surface):
        couleur = tuple(min(255, c + 40) for c in self.couleur) if self.survol else self.couleur
        pygame.draw.rect(surface, couleur, self.rect, border_radius=8)
        pygame.draw.rect(surface, BLANC, self.rect, width=2, border_radius=8)
        texte_surf = font.render(self.texte, True, BLANC)
        surface.blit(texte_surf, texte_surf.get_rect(center=self.rect.center))
    
    def verifier_survol(self, pos):
        self.survol = self.rect.collidepoint(pos)
        return self.survol

# Créer les boutons
btn_play = Bouton(100, 250, 120, 50, "Play", VERT)
btn_pause = Bouton(240, 250, 120, 50, "Pause", JAUNE)
btn_stop = Bouton(380, 250, 120, 50, "Stop", ROUGE)
btn_vol_up = Bouton(580, 250, 80, 50, "Vol+", BLEU)
btn_vol_down = Bouton(680, 250, 80, 50, "Vol-", BLEU)
boutons = [btn_play, btn_pause, btn_stop, btn_vol_up, btn_vol_down]

# ========== BOUCLE PRINCIPALE ==========
running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    # Mettre à jour le temps de lecture
    if en_lecture and not en_pause:
        temps_lecture += dt
        if temps_lecture >= duree_totale:
            temps_lecture = 0  # Boucler
    
    mouse_pos = pygame.mouse.get_pos()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            if event.key == pygame.K_SPACE:
                # Toggle play/pause
                if not en_lecture:
                    canal = melodie.play(-1)
                    canal.set_volume(volume)
                    en_lecture = True
                    en_pause = False
                elif en_pause:
                    canal.unpause()
                    en_pause = False
                else:
                    canal.pause()
                    en_pause = True
        
        if event.type == pygame.MOUSEBUTTONDOWN:
            if btn_play.rect.collidepoint(event.pos):
                if not en_lecture or en_pause:
                    if not en_lecture:
                        canal = melodie.play(-1)
                        canal.set_volume(volume)
                        en_lecture = True
                        temps_lecture = 0
                    else:
                        canal.unpause()
                    en_pause = False
            
            elif btn_pause.rect.collidepoint(event.pos):
                if en_lecture and not en_pause:
                    canal.pause()
                    en_pause = True
            
            elif btn_stop.rect.collidepoint(event.pos):
                if en_lecture:
                    melodie.stop()
                    en_lecture = False
                    en_pause = False
                    temps_lecture = 0
            
            elif btn_vol_up.rect.collidepoint(event.pos):
                volume = min(1.0, volume + 0.1)
                if canal:
                    canal.set_volume(volume)
            
            elif btn_vol_down.rect.collidepoint(event.pos):
                volume = max(0.0, volume - 0.1)
                if canal:
                    canal.set_volume(volume)
    
    # Vérifier le survol des boutons
    for btn in boutons:
        btn.verifier_survol(mouse_pos)
    
    # ---- Affichage ----
    ecran.fill(NOIR)
    
    # Titre
    titre = font.render("Lecteur de Musique", True, BLANC)
    ecran.blit(titre, (LARGEUR // 2 - titre.get_width() // 2, 30))
    
    # Nom du morceau
    nom = font_small.render("Mélodie procédurale (Do-Ré-Mi...)", True, GRIS_CLAIR)
    ecran.blit(nom, (LARGEUR // 2 - nom.get_width() // 2, 75))
    
    # État de lecture
    if en_lecture and not en_pause:
        etat = "En lecture"
        couleur_etat = VERT
    elif en_pause:
        etat = "En pause"
        couleur_etat = JAUNE
    else:
        etat = "Arrêté"
        couleur_etat = ROUGE
    
    etat_text = font.render(etat, True, couleur_etat)
    ecran.blit(etat_text, (LARGEUR // 2 - etat_text.get_width() // 2, 110))
    
    # Barre de progression
    barre_x, barre_y = 100, 170
    barre_w, barre_h = LARGEUR - 200, 20
    pygame.draw.rect(ecran, GRIS, (barre_x, barre_y, barre_w, barre_h))
    if duree_totale > 0:
        progression = (temps_lecture % duree_totale) / duree_totale
        pygame.draw.rect(ecran, VERT, (barre_x, barre_y, int(barre_w * progression), barre_h))
    pygame.draw.rect(ecran, BLANC, (barre_x, barre_y, barre_w, barre_h), 2)
    
    # Temps
    mins = int(temps_lecture) // 60
    secs = int(temps_lecture) % 60
    mins_tot = int(duree_totale) // 60
    secs_tot = int(duree_totale) % 60
    temps_text = font_small.render(f"{mins}:{secs:02d} / {mins_tot}:{secs_tot:02d}", True, BLANC)
    ecran.blit(temps_text, (LARGEUR // 2 - temps_text.get_width() // 2, barre_y + 30))
    
    # Boutons
    for btn in boutons:
        btn.dessiner(ecran)
    
    # Barre de volume
    vol_text = font_small.render(f"Volume: {int(volume * 100)}%", True, BLANC)
    ecran.blit(vol_text, (100, 330))
    pygame.draw.rect(ecran, GRIS, (250, 333, 300, 15))
    pygame.draw.rect(ecran, BLEU, (250, 333, int(300 * volume), 15))
    pygame.draw.rect(ecran, BLANC, (250, 333, 300, 15), 1)
    
    # Instructions
    instructions = font_small.render(
        "ESPACE: play/pause | Clic: boutons | ESC: quitter",
        True, (120, 120, 120)
    )
    ecran.blit(instructions, (LARGEUR // 2 - instructions.get_width() // 2, HAUTEUR - 40))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

## Pièges Courants

### 1. pre_init() après init()

```python
# ❌ ERREUR : pre_init n'a aucun effet après init
pygame.init()
pygame.mixer.pre_init(frequency=48000, buffer=256)  # Trop tard!

# ✅ CORRECT : pre_init AVANT init
pygame.mixer.pre_init(frequency=48000, buffer=256)
pygame.init()
```

### 2. Charger un son avant l'initialisation

```python
# ❌ ERREUR : mixer pas encore initialisé
son = pygame.mixer.Sound("tir.wav")
pygame.init()

# ✅ CORRECT : initialiser d'abord
pygame.init()
son = pygame.mixer.Sound("tir.wav")
```

### 3. Confondre Sound et music

```python
# ❌ ERREUR : Sound pour une musique de 5 minutes (mange la RAM)
musique = pygame.mixer.Sound("musique_5min.wav")  # Tout en mémoire!

# ✅ CORRECT : music pour les fichiers longs (streaming)
pygame.mixer.music.load("musique_5min.ogg")
pygame.mixer.music.play(-1)
```

### 4. MP3 qui ne boucle pas proprement

```python
# ❌ PROBLÈME : gap audible entre les boucles avec MP3
pygame.mixer.music.load("musique.mp3")
pygame.mixer.music.play(-1)  # Gap possible!

# ✅ SOLUTION : convertir en OGG
pygame.mixer.music.load("musique.ogg")
pygame.mixer.music.play(-1)  # Boucle parfaite
```

### 5. Trop de sons simultanés

```python
# ❌ PROBLÈME : saturation audio (cacophonie)
while running:
    for ennemi in ennemis:
        son_tir.play()  # 100 sons en même temps!

# ✅ SOLUTION : limiter avec un cooldown ou un pool
if temps_depuis_dernier_tir > 0.1:
    son_tir.play()
    temps_depuis_dernier_tir = 0
```

## Mini-Exercices

### Exercice 1 : Tableau de bord sonore

Créez un programme avec :
- 5 sons différents assignés aux touches 1-5 (fréquences différentes)
- Un affichage visuel montrant quel son est joué
- Contrôle du volume avec les flèches haut/bas
- Afficher le volume actuel à l'écran

In [ ]:
# Exercice 1 : À vous de coder!


### Exercice 2 : Jeu avec effets sonores

Créez un mini-jeu où :
- Un carré se déplace avec les flèches
- ESPACE joue un son de "tir" (son procédural aigu)
- Collision avec un objet joue un son de "collecte" (son procédural différent)
- Un son de "rebond" quand le carré touche les bords
- Tous les sons sont générés procéduralement

In [ ]:
# Exercice 2 : À vous de coder!


### Exercice 3 : Gestionnaire audio

Créez une classe `GestionnaireAudio` qui :
- Charge et stocke des sons par nom (dictionnaire)
- Gère un volume global et un volume par catégorie (effets, musique)
- Limite le nombre de sons identiques joués simultanément (pool)
- Affiche l'état de tous les canaux audio

In [ ]:
# Exercice 3 : À vous de coder!


## Solutions

### Solution Exercice 1

In [ ]:
%%writefile solution_ex1_tableau_sons.py
import pygame
import sys
import math
import array

pygame.mixer.pre_init(44100, -16, 1, 512)
pygame.init()

LARGEUR, HAUTEUR = 800, 400
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 1 : Tableau de bord sonore")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
GRIS = (60, 60, 60)


def creer_son(frequence, duree=0.3):
    """Créer un son sinusoïdal"""
    sample_rate = 44100
    buf = array.array('h')
    for i in range(int(sample_rate * duree)):
        t = i / sample_rate
        env = max(0, 1.0 - t / duree)  # Déclin
        val = int(20000 * math.sin(2 * math.pi * frequence * t) * env)
        buf.append(max(-32767, min(32767, val)))
    return pygame.mixer.Sound(buffer=buf)


# 5 sons avec des fréquences croissantes
config_sons = [
    ("Grave", 200, (180, 60, 60)),
    ("Bas", 330, (180, 120, 60)),
    ("Moyen", 440, (180, 180, 60)),
    ("Haut", 600, (60, 180, 60)),
    ("Aigu", 880, (60, 60, 180)),
]

sons = []
for nom, freq, couleur in config_sons:
    sons.append(creer_son(freq))

touches = [pygame.K_1, pygame.K_2, pygame.K_3, pygame.K_4, pygame.K_5]
volume = 0.7
actif = [0] * 5  # Timers d'animation

running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            # Jouer le son
            for i, touche in enumerate(touches):
                if event.key == touche:
                    sons[i].set_volume(volume)
                    sons[i].play()
                    actif[i] = 0.3
            # Volume
            if event.key == pygame.K_UP:
                volume = min(1.0, volume + 0.1)
            if event.key == pygame.K_DOWN:
                volume = max(0.0, volume - 0.1)
    
    # Décrémenter les timers
    for i in range(5):
        actif[i] = max(0, actif[i] - dt)
    
    # Affichage
    ecran.fill(NOIR)
    
    for i, (nom, freq, couleur) in enumerate(config_sons):
        x = 50 + i * 150
        y = 80
        c = couleur if actif[i] > 0 else GRIS
        pygame.draw.rect(ecran, c, (x, y, 120, 120), border_radius=10)
        pygame.draw.rect(ecran, BLANC, (x, y, 120, 120), 2, border_radius=10)
        
        txt = font.render(nom, True, BLANC)
        ecran.blit(txt, (x + 60 - txt.get_width() // 2, y + 30))
        txt2 = font.render(f"[{i+1}]", True, BLANC)
        ecran.blit(txt2, (x + 60 - txt2.get_width() // 2, y + 75))
    
    # Volume
    vol_txt = font.render(f"Volume: {int(volume * 100)}%", True, BLANC)
    ecran.blit(vol_txt, (50, 260))
    pygame.draw.rect(ecran, GRIS, (250, 265, 300, 20))
    pygame.draw.rect(ecran, (80, 200, 80), (250, 265, int(300 * volume), 20))
    
    info = font.render("Haut/Bas: volume", True, (120, 120, 120))
    ecran.blit(info, (50, HAUTEUR - 40))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

### Solution Exercice 2

In [ ]:
%%writefile solution_ex2_jeu_sons.py
import pygame
import sys
import math
import array
import random

pygame.mixer.pre_init(44100, -16, 1, 512)
pygame.init()

LARGEUR, HAUTEUR = 800, 600
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 2 : Jeu avec effets sonores")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 36)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
BLEU = (0, 100, 255)
JAUNE = (255, 215, 0)


def creer_son(freq, duree, type_onde="sin"):
    sample_rate = 44100
    buf = array.array('h')
    for i in range(int(sample_rate * duree)):
        t = i / sample_rate
        env = max(0, 1.0 - t / duree)
        if type_onde == "sin":
            val = math.sin(2 * math.pi * freq * t)
        else:
            # Onde carrée pour le tir
            val = 1 if math.sin(2 * math.pi * freq * t) > 0 else -1
        buf.append(max(-32767, min(32767, int(15000 * val * env))))
    return pygame.mixer.Sound(buffer=buf)


# Sons
son_tir = creer_son(800, 0.1, "square")
son_collecte = creer_son(600, 0.2, "sin")
son_rebond = creer_son(200, 0.15, "sin")

# Joueur
joueur_x, joueur_y = LARGEUR // 2, HAUTEUR // 2
taille_joueur = 30
vitesse = 300

# Objet à collecter
objet_x = random.randint(50, LARGEUR - 50)
objet_y = random.randint(50, HAUTEUR - 50)
score = 0

running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                son_tir.play()
    
    # Déplacement
    keys = pygame.key.get_pressed()
    ancien_x, ancien_y = joueur_x, joueur_y
    if keys[pygame.K_LEFT]: joueur_x -= vitesse * dt
    if keys[pygame.K_RIGHT]: joueur_x += vitesse * dt
    if keys[pygame.K_UP]: joueur_y -= vitesse * dt
    if keys[pygame.K_DOWN]: joueur_y += vitesse * dt
    
    # Rebond sur les bords
    rebond = False
    if joueur_x < 0:
        joueur_x = 0
        rebond = True
    elif joueur_x > LARGEUR - taille_joueur:
        joueur_x = LARGEUR - taille_joueur
        rebond = True
    if joueur_y < 0:
        joueur_y = 0
        rebond = True
    elif joueur_y > HAUTEUR - taille_joueur:
        joueur_y = HAUTEUR - taille_joueur
        rebond = True
    
    if rebond:
        son_rebond.play()
    
    # Collision avec l'objet
    rect_joueur = pygame.Rect(joueur_x, joueur_y, taille_joueur, taille_joueur)
    rect_objet = pygame.Rect(objet_x - 10, objet_y - 10, 20, 20)
    if rect_joueur.colliderect(rect_objet):
        son_collecte.play()
        score += 1
        objet_x = random.randint(50, LARGEUR - 50)
        objet_y = random.randint(50, HAUTEUR - 50)
    
    # Dessiner
    ecran.fill(NOIR)
    pygame.draw.rect(ecran, BLEU, (joueur_x, joueur_y, taille_joueur, taille_joueur))
    pygame.draw.circle(ecran, JAUNE, (objet_x, objet_y), 10)
    
    info = font.render(f"Score: {score} | ESPACE: tir", True, BLANC)
    ecran.blit(info, (10, 10))
    
    pygame.display.flip()

pygame.quit()
sys.exit()

### Solution Exercice 3

In [ ]:
%%writefile solution_ex3_gestionnaire.py
import pygame
import sys
import math
import array

pygame.mixer.pre_init(44100, -16, 1, 512)
pygame.init()

LARGEUR, HAUTEUR = 800, 500
ecran = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Exercice 3 : Gestionnaire Audio")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)


def generer_son(freq, duree=0.3, volume=0.5):
    sample_rate = 44100
    buf = array.array('h')
    for i in range(int(sample_rate * duree)):
        t = i / sample_rate
        env = max(0, 1.0 - t / duree)
        val = int(32767 * volume * math.sin(2 * math.pi * freq * t) * env)
        buf.append(max(-32767, min(32767, val)))
    return pygame.mixer.Sound(buffer=buf)


class GestionnaireAudio:
    def __init__(self):
        self.sons = {}  # nom -> Sound
        self.pools = {}  # nom -> [Sound, Sound, ...]
        self.pool_index = {}  # nom -> index courant
        self.volume_global = 1.0
        self.volume_effets = 0.8
        self.volume_musique = 0.5
    
    def ajouter_son(self, nom, son, max_simultanes=1):
        """Ajouter un son avec un nombre max d'instances simultanées"""
        if max_simultanes > 1:
            # Créer un pool
            self.pools[nom] = [son] + [
                pygame.mixer.Sound(son.get_raw()) for _ in range(max_simultanes - 1)
            ]
            self.pool_index[nom] = 0
        else:
            self.sons[nom] = son
        self._appliquer_volume(nom)
    
    def jouer(self, nom):
        """Jouer un son par son nom"""
        if nom in self.pools:
            index = self.pool_index[nom]
            self.pools[nom][index].play()
            self.pool_index[nom] = (index + 1) % len(self.pools[nom])
        elif nom in self.sons:
            self.sons[nom].play()
    
    def _appliquer_volume(self, nom=None):
        """Appliquer le volume sur un son ou tous les sons"""
        vol = self.volume_effets * self.volume_global
        if nom and nom in self.sons:
            self.sons[nom].set_volume(vol)
        elif nom and nom in self.pools:
            for s in self.pools[nom]:
                s.set_volume(vol)
        else:
            for s in self.sons.values():
                s.set_volume(vol)
            for pool in self.pools.values():
                for s in pool:
                    s.set_volume(vol)
    
    def set_volume_global(self, vol):
        self.volume_global = max(0.0, min(1.0, vol))
        self._appliquer_volume()
    
    def set_volume_effets(self, vol):
        self.volume_effets = max(0.0, min(1.0, vol))
        self._appliquer_volume()
    
    def get_etat_canaux(self):
        """Retourner l'état de tous les canaux"""
        nb = pygame.mixer.get_num_channels()
        etats = []
        for i in range(nb):
            canal = pygame.mixer.Channel(i)
            etats.append(canal.get_busy())
        return etats


# Créer le gestionnaire
audio = GestionnaireAudio()

# Ajouter des sons
audio.ajouter_son("tir", generer_son(800, 0.1), max_simultanes=4)
audio.ajouter_son("explosion", generer_son(150, 0.4), max_simultanes=2)
audio.ajouter_son("collecte", generer_son(600, 0.2))
audio.ajouter_son("alerte", generer_son(440, 0.5))

running = True
while running:
    dt = clock.tick(60) / 1000.0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            if event.key == pygame.K_1:
                audio.jouer("tir")
            if event.key == pygame.K_2:
                audio.jouer("explosion")
            if event.key == pygame.K_3:
                audio.jouer("collecte")
            if event.key == pygame.K_4:
                audio.jouer("alerte")
            if event.key == pygame.K_UP:
                audio.set_volume_global(audio.volume_global + 0.1)
            if event.key == pygame.K_DOWN:
                audio.set_volume_global(audio.volume_global - 0.1)
    
    # Affichage
    ecran.fill(NOIR)
    
    titre = font.render("Gestionnaire Audio", True, BLANC)
    ecran.blit(titre, (LARGEUR // 2 - titre.get_width() // 2, 20))
    
    # Sons disponibles
    y = 80
    noms = [("1: Tir (pool x4)", "tir"), ("2: Explosion (pool x2)", "explosion"),
            ("3: Collecte", "collecte"), ("4: Alerte", "alerte")]
    for label, nom in noms:
        texte = font.render(label, True, BLANC)
        ecran.blit(texte, (50, y))
        y += 35
    
    # Volume
    vol_txt = font.render(f"Volume global: {int(audio.volume_global * 100)}%", True, BLANC)
    ecran.blit(vol_txt, (50, 280))
    
    # État des canaux
    etats = audio.get_etat_canaux()
    y_canaux = 340
    ecran.blit(font.render("Canaux audio:", True, BLANC), (50, y_canaux))
    for i, actif in enumerate(etats):
        couleur = (0, 255, 0) if actif else (60, 60, 60)
        pygame.draw.rect(ecran, couleur, (50 + i * 40, y_canaux + 35, 30, 20))
        num = font.render(str(i), True, BLANC)
        ecran.blit(num, (55 + i * 40, y_canaux + 60))
    
    info = font.render("Haut/Bas: volume | ESC: quitter", True, (120, 120, 120))
    ecran.blit(info, (50, HAUTEUR - 40))
    
    pygame.display.flip()

pygame.quit()
sys.exit()